### 1. 理论知识

1. 不同于逻辑回归sigmoid函数，**y = 1/(1+e^(-(ax+b)))**，softmax函数预测多个概率，对应y属于每一类别的概率，如y=[0.7,0.2,0.1]
2. 对应的softmax公式如下：
3. ![Example Image](img/softmax公式1.jpg)
4. https://blog.csdn.net/xu380393916/article/details/102496419

### 2.数据准备

##### 2.1 导入数据集

In [16]:
# 导入模块
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

# 处理数据集
iris = load_iris()
X = pd.DataFrame(data=iris.data, columns=iris.feature_names)
y = pd.DataFrame(data=iris.target, columns=["label"])

# 特征重命名
map_ = {
    "sepal length (cm)": "花萼长度",
    "sepal width (cm)": "花萼宽度",
    "petal length (cm)": "花瓣长度",
    "petal width (cm)": "花瓣宽度",
}
X = X.rename(columns=map_)
X

,花萼长度,花萼宽度,花瓣长度,花瓣宽度
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


#### 2.2 数据处理

1. softmax回归基于线性回归y=ax+b，y = e^(ax+b)/sum(e^(ax+b))，**添加xn+1列=1**，以融合b到参数A，有y = e^(X·A)/sum(e^(X·A))

2. 由于softmax输出y对应每个标签的概率，如[0.7, 0.2, 0.1], 那么原始标签y也要处理成这样的形式，故此需要onehot编码。

**注意！！一定要对数据进行归一化，统一量纲，不然梯度下降大概率无法收敛！！**

In [17]:
from sklearn.preprocessing import Normalizer

# 数据归一化！！！！非常重要！！！！
X_1 = Normalizer().fit_transform(X)

# 添加补充列
X_1 = np.hstack((X_1, np.ones((len(X_1), 1))))

# 将y array化，方便后续计算，矩阵乘法@符号，和A.dot(B)形式都要求为array，否则只能用np.dot(A,B)
y = y.values

# 对标签进行one-hot编码,[0. 0. 0. 1.]，即每个标签在对应标签处为1，其余为0
n_classes = np.unique(y).shape[0]
n_samples = X_1.shape[0]
n_features = X_1.shape[1]

y_one_hot = np.zeros((n_samples, n_classes))
y_one_hot[np.arange(n_samples), y.T] = 1
y_one_hot

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

### 3. 算法实现

#### 3.1 梯度下降
**非凸函数，不可正规方程求解**

1. 首先，能够理解希望函数能输出每一个类别的概率，故此softmax函数有归一化操作：**X_1@A / np.sum(X_1@A,axis=1,keepdims=True)**

2. 为什么要将**X_1@A**指数化呢？**y = np.exp(X_1@A) / np.sum(np.exp(X_1@A),axis=1,keepdims=True)** 

3. 这么做主要有两个原因

    1. 通过将输出指数化，将假设函数的输出转换为正值。这对于创建一个有效的概率分布是必需的，因为概率值不能为负。
    
    2. 指数函数曲线呈现递增趋势，最重要的是斜率逐渐增大，也就是说在x轴上一个很小的变化，可以导致y轴上很大的变化。这有助于在计算概率时清晰地区分那些具有更高原始逻辑值的类别。

4. 交叉熵损失: ![Example Image](img/交叉熵损失1.jpg)

In [18]:
"""
对理论开始验证, 看下softmax函数每步操作的结果是否具有必要性以及原理性
"""

# 1. 首先，将y分别输出三类的概率，这样的思路是没问题的, 那么，初始化系数A
A = np.random.rand(n_features, n_classes)

# 很显然X·A的操作并不能实现输出概率化(3类加起来 = 1)
X_1 @ A

array([[0.84552377, 0.74903728, 1.52999726],
       [0.8229153 , 0.75277879, 1.5495243 ],
       [0.84539864, 0.74958422, 1.53420451],
       [0.84083251, 0.75909509, 1.56489725],
       [0.85503518, 0.7489956 , 1.52795113],
       [0.86887447, 0.75505298, 1.56777929],
       [0.87004417, 0.75261248, 1.55349934],
       [0.84266613, 0.75416084, 1.54671312],
       [0.83785721, 0.75804367, 1.56412446],
       [0.81856447, 0.7564913 , 1.54581977],
       [0.84402836, 0.74973185, 1.53082031],
       [0.84935586, 0.75928401, 1.56175255],
       [0.81713543, 0.75385755, 1.53861298],
       [0.84221167, 0.7449588 , 1.50685674],
       [0.84692225, 0.7345615 , 1.48395147],
       [0.88261838, 0.74317289, 1.52607541],
       [0.87284429, 0.74050299, 1.52393168],
       [0.85456098, 0.74883717, 1.54185843],
       [0.84469723, 0.75407277, 1.55561574],
       [0.86935275, 0.75053773, 1.54277765],
       [0.82498974, 0.75810338, 1.56180659],
       [0.87294526, 0.75100147, 1.55783432],
       [0.

In [19]:
# 归一化：有效果
X_1 @ A / np.sum(X_1 @ A, axis=1, keepdims=True)



array([[0.27060585, 0.23972581, 0.48966833],
       [0.26331449, 0.24087238, 0.49581313],
       [0.27016555, 0.23954597, 0.49028848],
       [0.26568058, 0.23985374, 0.49446567],
       [0.27300132, 0.23914429, 0.48785439],
       [0.27222879, 0.23656715, 0.49120405],
       [0.27392993, 0.23695703, 0.48911305],
       [0.26806279, 0.23990814, 0.49202907],
       [0.26514256, 0.23988531, 0.49497213],
       [0.2622868 , 0.24239714, 0.49531606],
       [0.27012534, 0.2399464 , 0.48992826],
       [0.26790244, 0.23949212, 0.49260544],
       [0.2627778 , 0.24242864, 0.49479355],
       [0.27220564, 0.24077319, 0.48702117],
       [0.27628124, 0.23962715, 0.48409161],
       [0.28003037, 0.23578818, 0.48418146],
       [0.27821698, 0.23603352, 0.4857495 ],
       [0.27169834, 0.2380846 , 0.49021706],
       [0.26778501, 0.23905534, 0.49315964],
       [0.27487954, 0.23731157, 0.4878089 ],
       [0.26232625, 0.24105805, 0.4966157 ],
       [0.27435743, 0.23603179, 0.48961079],
       [0.

In [20]:
# 根据实验，不做e的指数化不可行，尽管归一化能得到总和为1，但是特殊的A会导致预测值为负数(尽管3类别的预测值相加仍为1)，不能得到我们想要的概率。
prediction = np.exp(X_1 @ A) / np.sum(np.exp(X_1 @ A), axis=1, keepdims=True)
prediction

array([[0.25701987, 0.23337974, 0.50960039],
       [0.24997985, 0.23304785, 0.5169723 ],
       [0.25641231, 0.23298459, 0.51060311],
       [0.25098418, 0.23128541, 0.51773041],
       [0.25911212, 0.2330426 , 0.50784527],
       [0.25614979, 0.22859249, 0.51525772],
       [0.25840389, 0.22977306, 0.51182306],
       [0.2539865 , 0.2324734 , 0.51354009],
       [0.25058651, 0.23136364, 0.51804985],
       [0.24942588, 0.23441396, 0.51616016],
       [0.25658508, 0.23349573, 0.50991919],
       [0.25298923, 0.23119813, 0.51581264],
       [0.25024063, 0.23489653, 0.51486284],
       [0.25966425, 0.23560026, 0.50473549],
       [0.26423032, 0.2361484 , 0.49962128],
       [0.26504889, 0.23055024, 0.50440087],
       [0.26359721, 0.23092222, 0.50548057],
       [0.25720113, 0.23139694, 0.51140193],
       [0.25321449, 0.23127616, 0.51550935],
       [0.25981355, 0.23070717, 0.50947927],
       [0.24847338, 0.23239753, 0.51912909],
       [0.25848143, 0.22880728, 0.5127113 ],
       [0.

In [21]:
np.log(prediction)

array([[-1.35860189, -1.45508838, -0.6741284 ],
       [-1.38637498, -1.45651148, -0.65976598],
       [-1.36096856, -1.45678298, -0.67216269],
       [-1.38236536, -1.46410278, -0.65830062],
       [-1.3504944 , -1.45653399, -0.67757846],
       [-1.3619929 , -1.47581438, -0.66308807],
       [-1.35323147, -1.47066317, -0.6697763 ],
       [-1.37047416, -1.45897945, -0.66642717],
       [-1.38395106, -1.4637646 , -0.65768381],
       [-1.38859348, -1.45066665, -0.66133818],
       [-1.36029497, -1.45459148, -0.67350302],
       [-1.37440837, -1.46448022, -0.66201168],
       [-1.38533229, -1.44861017, -0.66385474],
       [-1.34836584, -1.44561871, -0.68372077],
       [-1.33093412, -1.44329487, -0.69390491],
       [-1.32784098, -1.46728647, -0.68438396],
       [-1.33333306, -1.46567436, -0.68224567],
       [-1.35789689, -1.4636207 , -0.67059944],
       [-1.37351835, -1.46414281, -0.66259984],
       [-1.347791  , -1.46660603, -0.67436611],
       [-1.39241954, -1.4593059 , -0.655

对数函数的特性可以放大小概率值的惩罚效果，而压缩大概率值的贡献。例如：

如果模型对正确类别的预测概率很高（如 0.9），
−
log

(
0.9
)
≈
0.1
−log(0.9)≈0.1，表示惩罚较小。
如果模型对正确类别的预测概率很低（如 0.01），
−
log
⁡
(
0.01
)
≈
4.6
−log(0.01)≈4.6，表示惩罚较大。
通过对概率取对数，模型可以更敏感地优化低概率错误，推动其改进。

In [22]:
# 通过交叉熵损失来计算损失函数：交叉熵损失用于衡量两个概率分布之间的差异
loss = -np.sum(y_one_hot * np.log(prediction))  # 固有公式: 交叉熵损失
loss = (
    loss / n_samples
)  # 运用于梯度求解时, 除以样本数, 能够平均化损失, 确保梯度更新的稳定性和一致性。
loss

np.float64(1.1527015161352128)

In [23]:
# 计算梯度
alpha = 0.1  # 学习率，用于控制每次更新的步长
lambd = 0.01  # L1正则化参数，用于防止过拟合，

"""实验证明，
1. 去掉L1正则化后, 函数拟合的快, 在相同的迭代次数时, 准确率更高, 但随着迭代次数增加时, A值就成了Nan,发生了过拟合, A系数变极端化。
2. 加上L1正则化后, 函数拟合的慢, 在相同的迭代次数时, 准确率较低，但随着迭代次数增加时, A值不会变成Nan值, 也就是不会发生极端变化。
3. 正则化有效的原因, 假如A为正数, A - lambd * A, A会加快变化; 假如A为负数, A - lambd * A, A会减慢变化
"""

for i in range(10000):

    # 计算预测值
    prediction = np.exp(X_1 @ A) / np.sum(np.exp(X_1 @ A), axis=1, keepdims=True)

    # 梯度整体 + lambd * A
    grad = (X_1.T @ (prediction - y_one_hot) / n_samples + lambd * A)  # 除以样本数, 确保梯度更新的稳定性和一致性。

    """
    在将偏置项融入特征矩阵时，通常会在特征矩阵的最后一列添加一个全为 1 的列, 注意是最后一列.
    这样，偏置项对应的权重就成为了参数矩阵 𝐴 的第 -1 行
    """
    grad[-1, :] = (grad[-1, :] - lambd * A[-1, :])  # 去掉正则化对偏置项的影响, 故偏置项 - lambd * A

    # 更新权重参数
    A = A - alpha * grad

In [24]:
result = X_1 @ A  # 注意这里的参数矩阵，必须转置才能进行矩阵相乘

# softmax函数将结果归一化，以此概率化
sum_exp = np.sum(np.exp(result), axis=1, keepdims=True)
prediction = np.exp(result) / sum_exp
prediction = pd.DataFrame(data=prediction, columns=np.unique(y))
prediction = prediction.idxmax(axis=1)

# 计算准确率
from sklearn.metrics import accuracy_score
accuracy_score(y, prediction)

0.84

### 4. 总结

1. 与sigmoid回归只预测 **正类概率** 不同, softmax回归同时 **预测标签的每一个类的概率**

2. 因此softmax的y要做onehot编码, 系数矩阵A的初始为: **A = np.random.rand(n_features, n_classes)**

3. 线性回归的梯度下降公式:  **gradients = X_1.T @ (X_1 @ A - y) / n**  (最小二乘)

4. 非线性回归梯度下降公式:  **gradients = X_1.T @ (X_1 @ A - y) / n**  (最小二乘)

5. Lasso回归梯度下降公式:   **gradients = X_1.T @ (X_1 @ A - y) / n + lambda_val * np.sign(A)**  (最小二乘)

6. Ridge回归梯度下降公式:   **gradients = X_1.T @ (X_1 @ A - y) / n + 2*lambda_val * A**  (最小二乘)

7. 逻辑sigmoid回归梯度下降公式:    **gradient = np.dot(X_1.T, (y_pre - y)) / n**   (最小二乘)

8. softmax回归梯度下降公式:  **grad = X_1.T@(prediction - y_one_hot ) / n_samples + lambd * A**  (交叉熵损失)

9. 从上可以看出, 梯度下降公式的一般原则:**X_1.T @ (预测值 - 真实值 )**